In [ ]:
!pip install scikit-learn
!pip install transformers
!pip install peft
!pip install 'accelerate>=0.26.0'
!pip install -U bitsandbytes
!pip install huggingface-hub
!pip install datasets
!pip install wandb
!pip install -U bitsandbytes

In [1]:
import json
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          Trainer,
                          pipeline,
                          DataCollatorForLanguageModeling,
                          PreTrainedTokenizer)
from peft import LoraConfig, get_peft_model
import huggingface_hub
import os
import logging
from tqdm import tqdm
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from typing import List
import wandb
from lora_llm import inference_Bedrock

d:\Thesis\thesis\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Thesis\thesis\.venv\lib\site-packages\pydantic\_internal\_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
d:\Thesis\thesis\.venv\lib\site-packages\pydantic\_internal\_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function

In [3]:
class_names = []
model_name = "us.anthropic.claude-3-sonnet-20240229-v1:0"

result = inference_Bedrock(model_name,
         "evasion_label",
         "clarity_label",
         "preprocessed_data/named_test_set_info.csv",
         added_name_summary=True)



  1%|          | 1/97 [00:01<01:47,  1.12s/it]

Right label: deflection


  2%|▏         | 2/97 [00:03<02:30,  1.58s/it]

Right label: deflection


  3%|▎         | 3/97 [00:05<02:52,  1.84s/it]

...

 99%|█████████▉| 96/97 [03:02<00:01,  1.79s/it]

Right label: deflection


100%|██████████| 97/97 [03:04<00:00,  1.90s/it]

Right label: deflection
Accuracy: 0.70
Accuracy for label indirect: 0.98
Accuracy for label direct reply: 0.17
Accuracy for label direct non-reply: 0.00

Classification Report:
                  precision    recall  f1-score   support

        indirect       0.69      0.98      0.81        64
    direct_reply       0.83      0.17      0.28        30
direct_non-reply       0.00      0.00      0.00         3

        accuracy                           0.70        97
       macro avg       0.51      0.38      0.36        97
    weighted avg       0.71      0.70      0.62        97


Confusion Matrix:
[[63  1  0]
 [25  5  0]
 [ 3  0  0]]



d:\Thesis\thesis\.venv\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\Thesis\thesis\.venv\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\Thesis\thesis\.venv\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [4]:
print(result)

['Accuracy: 0.70', 'Accuracy for label indirect: 0.98', 'Accuracy for label direct reply: 0.24', 'Accuracy for label direct non-reply: 0.17', 'Classification Report:                  precision    recall  f1-score   support\n\n        indirect       0.70      0.98      0.82       174\n    direct_reply       0.86      0.24      0.38        79\ndirect_non-reply       0.80      0.17      0.29        23\n\n       micro avg       0.72      0.70      0.71       276\n       macro avg       0.79      0.46      0.49       276\n    weighted avg       0.76      0.70      0.65       276\n', 'Confusion Matrix:[[170   3   0]\n [ 58  19   1]\n [ 14   0   4]]']
